# Google ADK Callbacks 완벽 가이드

이 노트북은 Google Agent Developer Kit (ADK)의 콜백(Callbacks) 기능에 대한 종합적인 튜토리얼이다. 콜백을 통해 에이전트의 생명주기(lifecycle) 각 단계에 개입하여 로깅, 가드(guarding), 캐싱, 프롬프트 수정 등 다양한 작업을 수행할 수 있다.

## 1. ADK 콜백 개요

ADK와 같은 에이전트 프레임워크의 핵심 실행 흐름은 다음과 같다:

```
입력 수신 → 모델 호출 → 도구 실행 → 출력 반환
```

실제 시스템에서는 이 각 단계에 개입(hook)해야 하는 경우가 많다. 예를 들어:
- **로깅**: 모든 요청/응답 기록
- **가드**: 유해한 입력/출력 차단
- **캐싱**: 동일한 요청에 대해 캐시된 응답 반환
- **프롬프트 수정**: 동적으로 프롬프트 변경
- **결과 변환**: 출력 형식 변환

콜백은 이런 요구사항을 충족시키는 "체크포인트"라고 생각하면 된다. ADK 프레임워크가 핵심 단계에서 자동으로 사용자 정의 함수를 호출해준다.

## 2. 콜백의 종류

ADK는 세 가지 주요 콜백 쌍을 제공한다:

| 콜백 유형 | Before 콜백 | After 콜백 | 설명 |
|----------|-------------|------------|------|
| **에이전트 콜백** | `before_agent_callback` | `after_agent_callback` | 에이전트의 전체 실행 전/후 |
| **모델 콜백** | `before_model_callback` | `after_model_callback` | LLM 요청/응답 전/후 |
| **도구 콜백** | `before_tool_callback` | `after_tool_callback` | 도구(함수) 실행 전/후 |

### 콜백의 동작 방식

콜백 함수는 두 가지 방식으로 동작할 수 있다:

1. **관찰 모드 (Observe)**: `None`을 반환하면 기본 동작이 계속 진행된다. 로깅이나 모니터링에 유용하다.
2. **오버라이드 모드 (Override)**: 특정 객체를 반환하면 기본 동작을 건너뛰거나 변경한다. 가드나 캐싱에 유용하다.

## 3. 환경 설정

먼저 필요한 패키지를 설치하고 임포트한다.

In [ ]:
# ADK 설치
%pip install google-adk -q

In [36]:
# 필요한 모듈 임포트
import asyncio
from typing import Optional, Dict, Any
from dotenv import load_dotenv

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai.types import Content, Part
from google.adk.agents.callback_context import CallbackContext
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse

In [37]:
load_dotenv()
MODEL = "gemini-2.5-flash"

## 4. 샘플 도구 정의

콜백 테스트를 위한 간단한 날씨 조회 도구를 정의한다. 이 도구는 에이전트가 호출할 수 있는 함수이다.

In [38]:
def get_weather(location: str) -> str:
    """
    주어진 위치의 날씨 정보를 반환하는 도구이다.
    
    Args:
        location: 날씨를 조회할 도시 이름
    
    Returns:
        해당 도시의 날씨 정보 문자열
    """
    # 실제 구현에서는 날씨 API를 호출한다
    weather_data = {
        "서울": "맑음, 기온 22°C",
        "부산": "흐림, 기온 25°C",
        "london": "비, 기온 15°C",
        "tokyo": "맑음, 기온 28°C"
    }
    return weather_data.get(location.lower(), f"{location}의 날씨: 정보 없음")

## 5. 에이전트 콜백 (Agent Callbacks)

에이전트 콜백은 에이전트의 전체 실행 사이클 전후에 실행된다.

### 반환 타입
- `None`: 기본 동작 허용
- `Content`: 에이전트 실행을 건너뛰거나 응답 수정

### 5.1 before_agent_callback

에이전트 로직이 시작되기 직전에 실행된다. 다음과 같은 용도로 활용할 수 있다:
- 요청 로깅
- 입력 검증
- 특정 조건에서 에이전트 실행 건너뛰기
- 세션 상태 확인

In [39]:
def before_agent_callback(callback_context: CallbackContext) -> Optional[Content]:
    """
    에이전트 실행 전에 호출되는 콜백이다.
    
    이 콜백에서는 다음 정보에 접근할 수 있다:
    - callback_context.agent_name: 에이전트 이름
    - callback_context.invocation_id: 호출 고유 ID
    - callback_context.state: 현재 세션 상태
    """
    print(f"▶ [before_agent_callback] 에이전트 실행 전")
    print(f"  에이전트 이름: {callback_context.agent_name}")
    print(f"  호출 ID: {callback_context.invocation_id}")
    print(f"  현재 상태: {callback_context.state.to_dict()}")
    
    # 테스트 플래그: True로 설정하면 에이전트 실행을 건너뛴다
    skip_agent = False
    
    if skip_agent:
        print(f" 에이전트 실행이 건너뛰어졌다")
        return Content(
            parts=[
                Part(
                    text=f"에이전트 '{callback_context.agent_name}' 실행이 "
                         f"'before_agent_callback'에 의해 건너뛰어졌다."
                )
            ],
            role="model",  # 오버라이딩 응답에 model 역할 할당
        )
    
    # None을 반환하면 기본 동작(에이전트 실행)이 진행된다
    return None

### 5.2 after_agent_callback

에이전트가 최종 결과를 생성한 직후에 실행된다. 다음과 같은 용도로 활용할 수 있다:
- 응답 로깅
- 출력 검증 및 필터링
- 응답에 추가 정보 덧붙이기
- 응답 형식 변환

In [40]:
def after_agent_callback(callback_context: CallbackContext) -> Optional[Content]:
    """
    에이전트 실행 후에 호출되는 콜백이다.
    
    에이전트의 응답을 수정하거나 추가 정보를 덧붙일 수 있다.
    """
    print(f"▶ [after_agent_callback] 에이전트 실행 후")
    print(f"  에이전트 이름: {callback_context.agent_name}")
    print(f"  호출 ID: {callback_context.invocation_id}")
    print(f"  현재 상태: {callback_context.state.to_dict()}")
    
    # 테스트 플래그: True로 설정하면 응답을 수정한다
    modify_response = False
    
    if modify_response:
        print(f" 에이전트 응답이 수정되었다")
        return Content(
            parts=[
                Part(
                    text="이 메시지는 'after_agent_callback'에 의해 추가되었다."
                )
            ],
            role="model",
        )
    
    # None을 반환하면 원래 응답이 그대로 반환된다
    return None

## 6. 모델 콜백 (Model Callbacks)

모델 콜백은 LLM(Large Language Model)과의 상호작용 전후에 실행된다.

### 반환 타입
- `None`: 기본 동작 허용
- `LlmResponse`: 모델 호출을 건너뛰거나 응답 수정

### 에이전트 콜백과의 차이점
- 에이전트 콜백: 전체 에이전트 실행 사이클에 대한 것
- 모델 콜백: 개별 LLM 호출에 대한 것 (에이전트가 여러 번 LLM을 호출할 수 있음)

### 6.1 before_model_callback

LLM에 요청을 보내기 직전에 실행된다. 다음과 같은 용도로 활용할 수 있다:
- 프롬프트 로깅
- 프롬프트 수정 (예: 시스템 프롬프트 추가)
- 캐시된 응답 반환
- 특정 조건에서 모델 호출 차단

In [41]:
def before_model_callback(
    callback_context: CallbackContext, 
    llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    모델 호출 전에 실행되는 콜백이다.
    
    Args:
        callback_context: 콜백 컨텍스트 (에이전트 정보, 세션 상태 등)
        llm_request: LLM에 보낼 요청 객체
    
    Returns:
        None: 기본 모델 호출 진행
        LlmResponse: 모델 호출을 건너뛰고 이 응답을 사용
    """
    print(f"▶ [before_model_callback] 모델 호출 전")
    print(f"  에이전트 이름: {callback_context.agent_name}")
    print(f"  호출 ID: {callback_context.invocation_id}")
    
    # llm_request에서 요청 내용을 확인할 수 있다
    # 예: llm_request.contents, llm_request.config 등
    
    # 테스트 플래그: True로 설정하면 모델 호출을 건너뛴다
    skip_model = False
    
    if skip_model:
        print(f" 모델 호출이 건너뛰어졌다")
        return LlmResponse(
            content=Content(
                parts=[Part(text="모델 호출이 'before_model_callback'에 의해 건너뛰어졌다.")],
                role="model",
            )
        )
    
    # None을 반환하면 기본 모델 호출이 진행된다
    return None

### 6.2 after_model_callback

LLM으로부터 응답을 받은 직후에 실행된다. 다음과 같은 용도로 활용할 수 있다:
- 응답 로깅
- 응답 변환 (예: 대문자로 변환, 형식 수정)
- 응답 검증 및 필터링
- 응답 캐싱

In [42]:
def after_model_callback(
    callback_context: CallbackContext, 
    llm_response: LlmResponse
) -> Optional[LlmResponse]:
    """
    모델 호출 후에 실행되는 콜백이다.
    
    Args:
        callback_context: 콜백 컨텍스트
        llm_response: LLM으로부터 받은 응답
    
    Returns:
        None: 원래 응답 그대로 사용
        LlmResponse: 수정된 응답으로 대체
    """
    print(f"▶ [after_model_callback] 모델 호출 후")
    print(f"  에이전트 이름: {callback_context.agent_name}")
    print(f"  호출 ID: {callback_context.invocation_id}")
    
    # 테스트 플래그: True로 설정하면 응답을 대문자로 변환한다
    modify_response = False
    
    if modify_response:
        # 응답 텍스트가 있는지 확인하고 대문자로 변환한다
        if llm_response.content and llm_response.content.parts:
            original_text = llm_response.content.parts[0].text
            print(f"  모델 응답이 대문자로 변환되었다")
            return LlmResponse(
                content=Content(
                    parts=[
                        Part(
                            text=f"[수정됨 by after_model_callback] {original_text.upper()}"
                        )
                    ],
                    role="model",
                )
            )
    
    # None을 반환하면 원래 응답이 그대로 사용된다
    return None

## 7. 도구 콜백 (Tool Callbacks)

도구 콜백은 에이전트가 도구(함수)를 실행할 때 전후로 실행된다.

### 반환 타입
- `None`: 기본 동작 허용
- `Dict` 또는 값: 도구 실행을 건너뛰거나 응답 수정

### 도구 콜백의 특징
- 도구별로 다른 동작을 정의할 수 있다
- 도구 인자(args)에 접근하여 조건부 로직을 구현할 수 있다
- 도구 응답을 수정하거나 대체할 수 있다

### 7.1 before_tool_callback

도구가 실행되기 직전에 실행된다. 다음과 같은 용도로 활용할 수 있다:
- 도구 호출 로깅
- 인자 검증
- 캐시된 결과 반환
- 특정 조건에서 도구 실행 차단

In [43]:
def before_tool_callback(
    tool: BaseTool, 
    args: Dict[str, Any], 
    tool_context: ToolContext
) -> Optional[Dict]:
    """
    도구 실행 전에 호출되는 콜백이다.
    
    Args:
        tool: 실행될 도구 객체
        args: 도구에 전달될 인자들
        tool_context: 도구 컨텍스트 (에이전트 정보 등)
    
    Returns:
        None: 기본 도구 실행 진행
        Dict 또는 값: 도구 실행을 건너뛰고 이 값을 결과로 사용
    """
    print(f"▶ [before_tool_callback] 도구 실행 전")
    print(f"  에이전트 이름: {tool_context.agent_name}")
    print(f"  호출 ID: {tool_context.invocation_id}")
    print(f"  도구 이름: {tool.name}")
    print(f"  인자: {args}")
    
    # 테스트 플래그: True로 설정하면 특정 조건에서 도구 실행을 건너뛴다
    use_mock = False
    
    if use_mock:
        # 예: 런던 날씨 요청에 대해 모의 응답 반환
        if tool.name == "get_weather" and args.get("location", "").lower() == "london":
            mock_response = "런던의 날씨는 항상 비가 오고 우울하다."
            print(f" 런던에 대한 도구 실행이 건너뛰어지고 모의 응답이 반환된다: {mock_response}")
            return mock_response
    
    # None을 반환하면 기본 도구 실행이 진행된다
    return None

### 7.2 after_tool_callback

도구 실행 후에 실행된다. 다음과 같은 용도로 활용할 수 있다:
- 도구 결과 로깅
- 결과 변환 또는 포맷팅
- 결과 검증
- 결과 캐싱

In [44]:
def after_tool_callback(
    tool: BaseTool, 
    args: Dict[str, Any], 
    tool_context: ToolContext, 
    tool_response: Dict
) -> Optional[Dict]:
    """
    도구 실행 후에 호출되는 콜백이다.
    
    Args:
        tool: 실행된 도구 객체
        args: 도구에 전달된 인자들
        tool_context: 도구 컨텍스트
        tool_response: 도구가 반환한 결과
    
    Returns:
        None: 원래 결과 그대로 사용
        Dict 또는 값: 수정된 결과로 대체
    """
    print(f"▶ [after_tool_callback] 도구 실행 후")
    print(f"  에이전트 이름: {tool_context.agent_name}")
    print(f"  호출 ID: {tool_context.invocation_id}")
    print(f"  도구 이름: {tool.name}")
    print(f"  원본 응답: {tool_response}")
    
    # 테스트 플래그: True로 설정하면 응답을 수정한다
    modify_response = False
    
    if modify_response:
        if tool.name == "get_weather":
            modified_response = "날씨는 항상 비가 오고 우울하다. (수정됨)"
            print(f" 날씨 도구 응답이 수정되었다: {modified_response}")
            return modified_response
    
    # None을 반환하면 원래 결과가 그대로 사용된다
    return None

## 8. 에이전트 생성 및 콜백 등록

이제 모든 콜백을 등록한 에이전트를 생성한다. 에이전트 생성 시 콜백 함수들을 인자로 전달하면 된다.

In [45]:
# 모든 콜백이 등록된 에이전트 생성
root_agent = Agent(
    model=MODEL,  # 사용할 LLM 모델
    name="callback_demo_agent",
    description="콜백 기능을 시연하는 에이전트이다.",
    instruction="사용자의 질문에 최선을 다해 답변하라. 날씨 정보가 필요하면 get_weather 도구를 사용하라.",
    tools=[get_weather],  # 사용 가능한 도구 목록
    
    # 에이전트 콜백
    before_agent_callback=before_agent_callback,
    after_agent_callback=after_agent_callback,
    
    # 모델 콜백
    before_model_callback=before_model_callback,
    after_model_callback=after_model_callback,
    
    # 도구 콜백
    before_tool_callback=before_tool_callback,
    after_tool_callback=after_tool_callback,
)

## 9. 에이전트 실행 및 테스트

Runner를 사용하여 에이전트를 실행하고 콜백 동작을 확인한다.

### 중요: 비동기 처리

ADK의 `InMemorySessionService.create_session()`과 `Runner.run_async()`는 **비동기(async) 함수**이다. 따라서 `await` 키워드를 사용해야 한다. Jupyter Notebook에서는 셀 내에서 직접 `await`를 사용할 수 있다.

In [46]:
# 상수 정의
APP_NAME = "callback_demo"
USER_ID = "test_user"
SESSION_ID = "session_001"

# 세션 서비스 생성
session_service = InMemorySessionService()

# 러너 생성
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# 세션 생성 (비동기 함수이므로 await 필요)
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"세션 생성 완료!")
print(f"  세션 ID: {session.id}")
print(f"  앱 이름: {session.app_name}")
print(f"  사용자 ID: {session.user_id}")

세션 생성 완료!
  세션 ID: session_001
  앱 이름: callback_demo
  사용자 ID: test_user


### 9.1 에이전트 호출 헬퍼 함수

에이전트를 호출하고 응답을 출력하는 헬퍼 함수를 정의한다.

In [47]:
async def call_agent(user_message: str):
    """
    에이전트에게 메시지를 보내고 응답을 출력하는 헬퍼 함수이다.
    
    Args:
        user_message: 사용자 메시지
    """
    print(f"\n{'='*60}")
    print(f"사용자: {user_message}")
    print(f"{'='*60}\n")
    
    # Content 객체 생성
    content = Content(
        parts=[Part(text=user_message)],
        role="user"
    )
    
    # 에이전트 실행 (비동기 제너레이터)
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session.id,
        new_message=content
    ):
        # 최종 응답 이벤트만 출력
        if event.is_final_response():
            if event.content and event.content.parts:
                response_text = event.content.parts[0].text
                print(f"\n{'='*60}")
                print(f"에이전트 응답: {response_text}")
                print(f"{'='*60}")

### 9.2 간단한 질문 테스트

도구를 사용하지 않는 간단한 질문으로 에이전트와 모델 콜백을 테스트한다.

In [48]:
# 테스트 1: 간단한 질문 (도구 사용 없음)
print("\n" + "#"*60)
print("# 테스트 1: 간단한 질문 (도구 사용 없음)")
print("#"*60)

await call_agent("안녕하세요! 오늘 기분이 어떠세요?")


############################################################
# 테스트 1: 간단한 질문 (도구 사용 없음)
############################################################

사용자: 안녕하세요! 오늘 기분이 어떠세요?

▶ [before_agent_callback] 에이전트 실행 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-d68331d0-e1ac-497d-a2fb-e0eff3e74128
  현재 상태: {}
▶ [before_model_callback] 모델 호출 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-d68331d0-e1ac-497d-a2fb-e0eff3e74128
▶ [after_model_callback] 모델 호출 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-d68331d0-e1ac-497d-a2fb-e0eff3e74128

에이전트 응답: 안녕하세요! 저는 잘 지내고 있습니다. 궁금한 점이 있으시면 언제든지 말씀해주세요. 😊
▶ [after_agent_callback] 에이전트 실행 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-d68331d0-e1ac-497d-a2fb-e0eff3e74128
  현재 상태: {}


### 9.3 도구 사용 테스트

날씨 질문으로 도구 콜백까지 포함한 전체 콜백 흐름을 테스트한다.

In [49]:
# 테스트 2: 도구 사용 질문
print("\n" + "#"*60)
print("# 테스트 2: 도구 사용 질문")
print("#"*60)

await call_agent("서울 날씨 어때?")


############################################################
# 테스트 2: 도구 사용 질문
############################################################

사용자: 서울 날씨 어때?

▶ [before_agent_callback] 에이전트 실행 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4
  현재 상태: {}
▶ [before_model_callback] 모델 호출 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4


▶ [after_model_callback] 모델 호출 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4
▶ [before_tool_callback] 도구 실행 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4
  도구 이름: get_weather
  인자: {'location': '서울'}
▶ [after_tool_callback] 도구 실행 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4
  도구 이름: get_weather
  원본 응답: 맑음, 기온 22°C
▶ [before_model_callback] 모델 호출 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4
▶ [after_model_callback] 모델 호출 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4

에이전트 응답: 서울은 맑고 기온은 22°C입니다.
▶ [after_agent_callback] 에이전트 실행 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-546c562d-4e1c-40da-a944-cb6361e701c4
  현재 상태: {}


### 9.4 여러 도시 날씨 테스트

In [50]:
# 테스트 3: 영어 도시 이름
print("\n" + "#"*60)
print("# 테스트 3: 영어 도시 이름으로 날씨 조회")
print("#"*60)

await call_agent("What's the weather like in London?")


############################################################
# 테스트 3: 영어 도시 이름으로 날씨 조회
############################################################

사용자: What's the weather like in London?

▶ [before_agent_callback] 에이전트 실행 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-0fc8b621-0da7-48ce-82ed-da8e5d6cd5bf
  현재 상태: {}
▶ [before_model_callback] 모델 호출 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-0fc8b621-0da7-48ce-82ed-da8e5d6cd5bf
▶ [after_model_callback] 모델 호출 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-0fc8b621-0da7-48ce-82ed-da8e5d6cd5bf
▶ [before_tool_callback] 도구 실행 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-0fc8b621-0da7-48ce-82ed-da8e5d6cd5bf
  도구 이름: get_weather
  인자: {'location': '런던'}
▶ [after_tool_callback] 도구 실행 후
  에이전트 이름: callback_demo_agent
  호출 ID: e-0fc8b621-0da7-48ce-82ed-da8e5d6cd5bf
  도구 이름: get_weather
  원본 응답: 런던의 날씨: 정보 없음
▶ [before_model_callback] 모델 호출 전
  에이전트 이름: callback_demo_agent
  호출 ID: e-0fc8b621-0da7-48ce-82ed-da8e5d6cd5bf
▶ [after_model_callback] 모델 호출 

## 10. 실용적인 콜백 활용 예제

아래는 실제 프로덕션 환경에서 유용하게 사용할 수 있는 콜백 예제들이다.

### 10.1 입력 검증 콜백 (Guard)

유해하거나 부적절한 입력을 차단하는 가드 콜백이다.

In [51]:
def input_guard_callback(callback_context: CallbackContext) -> Optional[Content]:
    """
    입력을 검증하고 부적절한 요청을 차단하는 콜백이다.
    실제 구현에서는 세션 상태나 이전 이벤트에서 사용자 입력을 확인한다.
    """
    # 차단할 키워드 목록
    blocked_keywords = ["해킹", "불법", "비밀번호"]
    
    # 세션 상태 확인
    state = callback_context.state.to_dict()
    print(f"  [Guard] 입력 검증 중...")
    
    # 실제로는 여기서 사용자 입력 텍스트를 검사한다
    # 예시: state에서 마지막 사용자 메시지 확인
    
    return None  # 검증 통과

print("입력 검증 콜백이 정의되었다.")

입력 검증 콜백이 정의되었다.


### 10.2 응답 캐싱 콜백

동일한 요청에 대해 캐시된 응답을 반환하는 콜백이다.

In [52]:
import hashlib

# 간단한 인메모리 캐시
response_cache: Dict[str, LlmResponse] = {}

def caching_before_model_callback(
    callback_context: CallbackContext, 
    llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    캐시된 응답이 있으면 모델 호출을 건너뛰고 캐시된 응답을 반환한다.
    """
    # 요청을 캐시 키로 변환
    if llm_request.contents:
        cache_key = hashlib.md5(str(llm_request.contents).encode()).hexdigest()
        
        if cache_key in response_cache:
            print("  📦 캐시 히트! 캐시된 응답을 반환한다.")
            return response_cache[cache_key]
    
    return None

def caching_after_model_callback(
    callback_context: CallbackContext, 
    llm_response: LlmResponse
) -> Optional[LlmResponse]:
    """
    모델 응답을 캐시에 저장한다.
    """
    print("  💾 응답이 캐시에 저장되었다.")
    # 실제 구현에서는 여기서 캐시에 저장
    return None

print("캐싱 콜백이 정의되었다.")

캐싱 콜백이 정의되었다.


### 10.3 구조화된 로깅 콜백

모든 상호작용을 JSON 형태로 로깅하는 콜백이다.

In [53]:
import json
from datetime import datetime

def structured_logging_before_agent(callback_context: CallbackContext) -> Optional[Content]:
    """
    에이전트 호출을 구조화된 JSON 형태로 로깅한다.
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "event_type": "agent_start",
        "agent_name": callback_context.agent_name,
        "invocation_id": callback_context.invocation_id,
        "state_keys": list(callback_context.state.to_dict().keys())
    }
    print(f"📝 LOG: {json.dumps(log_entry, ensure_ascii=False)}")
    return None

def structured_logging_after_agent(callback_context: CallbackContext) -> Optional[Content]:
    """
    에이전트 완료를 구조화된 JSON 형태로 로깅한다.
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "event_type": "agent_end",
        "agent_name": callback_context.agent_name,
        "invocation_id": callback_context.invocation_id
    }
    print(f"📝 LOG: {json.dumps(log_entry, ensure_ascii=False)}")
    return None

print("구조화된 로깅 콜백이 정의되었다.")

구조화된 로깅 콜백이 정의되었다.


### 10.4 도구 실행 시간 측정 콜백

도구 실행 시간을 측정하여 성능 모니터링에 활용할 수 있는 콜백이다.

In [54]:
import time

# 도구 시작 시간을 저장하는 딕셔너리
tool_start_times: Dict[str, float] = {}

def timing_before_tool_callback(
    tool: BaseTool, 
    args: Dict[str, Any], 
    tool_context: ToolContext
) -> Optional[Dict]:
    """
    도구 실행 시작 시간을 기록한다.
    """
    key = f"{tool_context.invocation_id}_{tool.name}"
    tool_start_times[key] = time.time()
    print(f"  ⏱️ [{tool.name}] 실행 시작")
    return None

def timing_after_tool_callback(
    tool: BaseTool, 
    args: Dict[str, Any], 
    tool_context: ToolContext, 
    tool_response: Dict
) -> Optional[Dict]:
    """
    도구 실행 시간을 계산하고 출력한다.
    """
    key = f"{tool_context.invocation_id}_{tool.name}"
    if key in tool_start_times:
        elapsed = time.time() - tool_start_times[key]
        print(f"  ⏱️ [{tool.name}] 실행 완료: {elapsed:.3f}초")
        del tool_start_times[key]
    return None

print("도구 실행 시간 측정 콜백이 정의되었다.")

도구 실행 시간 측정 콜백이 정의되었다.


## 11. 콜백 실행 순서 정리

에이전트가 도구를 사용하는 전체 흐름에서 콜백이 실행되는 순서는 다음과 같다:

```
1. before_agent_callback      ← 에이전트 시작
   │
   ├─▶ 2. before_model_callback   ← 첫 번째 LLM 호출 (도구 선택)
   │       │
   │       └─▶ 3. after_model_callback
   │
   ├─▶ 4. before_tool_callback    ← 도구 실행
   │       │
   │       └─▶ 5. after_tool_callback
   │
   ├─▶ 6. before_model_callback   ← 두 번째 LLM 호출 (최종 응답 생성)
   │       │
   │       └─▶ 7. after_model_callback
   │
   └─▶ 8. after_agent_callback    ← 에이전트 종료
```

**참고**: 모델과 도구 콜백은 에이전트의 로직에 따라 여러 번 호출될 수 있다.

## 12. 콜백 오버라이드 테스트

콜백에서 값을 반환하여 기본 동작을 오버라이드하는 예제를 테스트한다.

In [56]:
# 에이전트 실행을 건너뛰는 before_agent_callback
def skip_agent_callback(callback_context: CallbackContext) -> Optional[Content]:
    """
    항상 에이전트 실행을 건너뛰고 고정된 응답을 반환한다.
    """
    print(f"▶ [skip_agent_callback] 에이전트 실행을 건너뛴다!")
    return Content(
        parts=[Part(text="이 응답은 에이전트를 거치지 않고 콜백에서 직접 생성되었다.")],
        role="model",
    )

# 테스트용 에이전트 생성
skip_test_agent = Agent(
    model=MODEL,
    name="skip_test_agent",
    description="에이전트 건너뛰기 테스트용",
    instruction="사용자 질문에 답변하라.",
    before_agent_callback=skip_agent_callback,
)

print("에이전트 건너뛰기 테스트 에이전트가 생성되었다.")

에이전트 건너뛰기 테스트 에이전트가 생성되었다.


In [57]:
# 테스트 실행
skip_session_service = InMemorySessionService()
skip_runner = Runner(
    agent=skip_test_agent,
    app_name="skip_test",
    session_service=skip_session_service
)

skip_session = await skip_session_service.create_session(
    app_name="skip_test",
    user_id="test_user",
    session_id="skip_session"
)

print("\n" + "#"*60)
print("# 에이전트 건너뛰기 테스트")
print("#"*60)

content = Content(parts=[Part(text="복잡한 질문입니다.")], role="user")

async for event in skip_runner.run_async(
    user_id="test_user",
    session_id=skip_session.id,
    new_message=content
):
    if event.is_final_response() and event.content:
        print(f"\n응답: {event.content.parts[0].text}")


############################################################
# 에이전트 건너뛰기 테스트
############################################################
▶ [skip_agent_callback] 에이전트 실행을 건너뛴다!

응답: 이 응답은 에이전트를 거치지 않고 콜백에서 직접 생성되었다.


## 13. 주의사항 및 모범 사례

### 주의사항

1. **비동기 처리**: ADK의 세션 생성과 에이전트 실행은 비동기 함수이다. 반드시 `await`를 사용해야 한다.

2. **성능 고려**: 콜백에서 무거운 작업(예: 외부 API 호출, 복잡한 연산)을 수행하면 전체 응답 시간이 늘어난다.

3. **예외 처리**: 콜백 내에서 발생하는 예외는 적절히 처리해야 한다. 처리되지 않은 예외는 전체 에이전트 실행을 중단시킬 수 있다.

4. **상태 관리**: 콜백 간에 상태를 공유할 때는 스레드 안전성을 고려해야 한다.

### 모범 사례

1. **단일 책임 원칙**: 각 콜백은 한 가지 목적만 수행하도록 설계한다.

2. **충분한 로깅**: 디버깅을 위해 콜백에서 충분한 로깅을 추가한다.

3. **테스트 작성**: 콜백 로직에 대한 단위 테스트를 작성한다.

4. **문서화**: 각 콜백의 목적과 동작을 명확히 문서화한다.

5. **타임아웃 설정**: 외부 서비스 호출 시 적절한 타임아웃을 설정한다.

## 14. 결론

이 튜토리얼에서는 Google ADK의 콜백 시스템에 대해 살펴보았다. 콜백을 활용하면 에이전트의 동작을 세밀하게 제어하고 확장할 수 있다.

### 핵심 포인트 요약

| 콜백 유형 | 반환 타입 (오버라이드) | 주요 용도 |
|----------|----------------------|----------|
| 에이전트 콜백 | `Content` | 전체 실행 제어, 입력/출력 검증 |
| 모델 콜백 | `LlmResponse` | LLM 호출 제어, 캐싱, 프롬프트 수정 |
| 도구 콜백 | `Dict` 또는 값 | 도구 실행 제어, 모킹, 결과 변환 |

### 기억해야 할 사항

- `None` 반환: 기본 동작 진행
- 특정 객체 반환: 기본 동작 오버라이드
- 비동기 함수 사용 시 `await` 필수

### 추가 자료

- [ADK 공식 문서 - Callbacks](https://google.github.io/adk-docs/callbacks/)
- [ADK GitHub Repository](https://github.com/google/adk-python)
- [ADK 세션 관리](https://google.github.io/adk-docs/sessions/session/)